In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras==2.13.1
!pip install tensorflow==2.13.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires keras>=3.2.0, but you have keras 2.13.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4

In [ ]:

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


import keras
from keras.models import Model,Sequential
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, BatchNormalization, Dropout
from keras import regularizers
from keras.optimizers import SGD
# from clr_callback import *
%matplotlib inline


In [ ]:
import keras
import tensorflow
print(keras.__version__)
print(tensorflow.__version__)

2.13.1
2.13.1


In [ ]:
# Useful Constants

# Output classes to learn how to classify
LABELS = [
    "RUNNING",
    "STANDING",
    "Walking",
    "Strong kicking",
    "Passing",

]
DATASET_PATH = "/content/"


# x_train_path = DATASET_PATH + "body16Raw_Train_X.csv"
x_train_path = DATASET_PATH + "Raw_Train_X.csv"
# x_test_path = DATASET_PATH + "body16Raw_Test_X.csv"
x_test_path = DATASET_PATH + "Raw_Test_X.csv"

# x_test_path = DATASET_PATH + "testx.csv" #單場景測試

y_train_path = DATASET_PATH + "Raw_Train_Y.csv"
y_test_path = DATASET_PATH + "Raw_Test_Y.csv"

# y_test_path = DATASET_PATH + "testy.csv" #單場景測試

n_steps = 16    # 32 frames per series

In [ ]:
def load_X(path):
    data=pd.read_csv(path,header=None).values
    blocks = int(len(data) / n_steps)
    data=np.array(np.split(data,blocks))
    return data

def load_y(path):
    data=pd.read_csv(path,header=None).values
    return data-1

In [ ]:
#Load the data
X_train = load_X(x_train_path)
X_test = load_X(x_test_path)
y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

In [ ]:
# Input Data
training_data_count = len(X_train)  # 4519 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 1197 test series
n_input = len(X_train[0][0])
n_hidden = 512 # Hidden layer num of features
n_classes = 5  #number of action classes

batch_size = 16


In [ ]:
y_train_one_hot = to_categorical(y_train, num_classes=5)
y_test_one_hot = to_categorical(y_test, 5)

train_size = X_train.shape[0] - X_train.shape[0] % batch_size
test_size = X_test.shape[0] - X_test.shape[0] % batch_size

In [ ]:
model = Sequential([
   Dense(n_hidden, activation='relu'),
   BatchNormalization(),
   LSTM(n_hidden, return_sequences=True,  unit_forget_bias=1.0,dropout=0.2),
   LSTM(n_hidden,  unit_forget_bias=1.0),
   BatchNormalization(),
   Dense(n_classes,
       activation='softmax')
])

In [ ]:
# #LR range test
# clr = CyclicLR(base_lr=0.0001, max_lr=1, step_size=np.ceil(X_train.shape[0]/(batch_size)), mode='triangular')

In [ ]:
# model.compile(
#    optimizer=SGD(),
#    metrics=['accuracy'],
#    loss='categorical_crossentropy'
# )


In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)  # Specify the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)


# from tensorflow.keras.optimizers import SGD

# optimizer = SGD(learning_rate=0.01, momentum=0.9)  # 設置學習率和動量

# from tensorflow.keras.optimizers import RMSprop

# optimizer = RMSprop(learning_rate=0.001, rho=0.9)

history = model.fit(X_train[:train_size,:,:], y_train_one_hot[:train_size,:], epochs=50, batch_size=batch_size)


# 訓練集準確率
train_loss, train_accuracy = model.evaluate(X_train, y_train_one_hot, verbose=0)
print(f"訓練集準確率：{train_accuracy:.4f}")

# 測試集準確率
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=0)
print(f"測試集準確率：{test_accuracy:.4f}")


Epoch 1/50
31/31 [==============================] - 33s 1s/step - loss: 1.8919 - accuracy: 0.5605
Epoch 2/50
31/31 [==============================] - 26s 831ms/step - loss: 0.7233 - accuracy: 0.7641
Epoch 3/50
31/31 [==============================] - 25s 818ms/step - loss: 0.5469 - accuracy: 0.8044
Epoch 4/50
31/31 [==============================] - 27s 880ms/step - loss: 0.4770 - accuracy: 0.8306
Epoch 5/50
31/31 [==============================] - 25s 808ms/step - loss: 0.3951 - accuracy: 0.8649
Epoch 6/50
31/31 [==============================] - 26s 826ms/step - loss: 0.2790 - accuracy: 0.8851
Epoch 7/50
31/31 [==============================] - 25s 815ms/step - loss: 0.3372 - accuracy: 0.8790
Epoch 8/50
31/31 [==============================] - 27s 850ms/step - loss: 0.3140 - accuracy: 0.8992
Epoch 9/50
31/31 [==============================] - 26s 791ms/step - loss: 0.3505 - accuracy: 0.8972
Epoch 10/50
31/31 [==============================] - 25s 795ms/step - loss: 0.2439 - accuracy:

In [ ]:
# # Save the model
model.save('/content/adam091304.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model, Sequential
from sklearn.metrics import confusion_matrix
model = load_model('/content/adam091304.h5')

# Generate predictions on the test set
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Get the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Normalize the confusion matrix
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


# Plot the confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=LABELS, yticklabels=LABELS,
       title='Confusion matrix',
       ylabel='True label',
       xlabel='Predicted label')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
fmt = '.4f'
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], fmt),
               ha="center", va="center", color="white" if cm[i, j] > thresh else "black")
fig.tight_layout()
plt.show()

